In [1]:
# SCRIPT: SAIHEBRO.py
# AUTHOR: Henrik Schmidt
# DATE: 13.03.2022
#==================================================================================================================
# This script contains functions to clean-up SAIHEBRO meteorogical parameters csv-files from redundant whitespaces 
# and merge different parameter-files stationwise. For this purpose csv-files and one or more text-files 
# are needed. The text-files should contain the station-numbers where each station should occupie an extra line.
# E.g.:
# P080
# P081
# ...
#==================================================================================================================
import matplotlib as plt
import pandas as pd
import os, re, glob
import numpy as np

#==================================================================================================================
# Function to add a sign in as first element in a file
def add_firstsign(path,sign='#'):
    os.chdir(path) # changing active path
    file_list = glob.glob('*.{}'.format('csv')) # list all csv files in the specified path
    for file in file_list: # iterate through the filelist
        file_read = open(file,'r') # read the lines out of the file
        lines = file_read.readlines()
        first_line = sign + lines[0] # add the specified sign, standard #
        file_read.close() 
        file_write = open(file,'w') # write the new first line and all the other lines (file will be empty otherwise)
        for i in range(len(lines)):
            if i == 0:
                file_write.write(first_line) # write the first line in the first place
            else:
                file_write.write(lines[i]) # write all the other lines
        file_write.close()
    print('Added # in first row...') # Progress
#==================================================================================================================
# Function to extract all the station numbers as an array
def array_stations(path):
    file = open(path,'r') # Open the file
    array_stations = file.readlines() # Read out each line and place them in an array
    # Check for ',' in the array (E.g.: P080,TEMPE OR P080)
    test = re.search(',', array_stations[0])
    if test != None: # E.g.: P080,TEMPE in text-file
        for value, line in enumerate(array_stations,0):
            array_stations[value] = line.split(',') # Split the line by ','
            array_stations[value] = array_stations[value][0] # Just keep the first element
            # E.g.: Returns only P080
    else: # E.g.: P080 in text-file
        array_stations = [line.strip() for line in file.readlines()] # Keep the line and just erase whitespaces
    file = None # Close the file, so no read/write conflict occurs
    
    return array_stations # Return the array
#==================================================================================================================
# Function to clean the saihebro csv-files from redundant whitespaces
# BEFORE CLEANING E.g.: date     ;  measurment_a         ;  measurement_b        ;....
# AFTER CLEANING E.g.: date; measurment_a; measurement_b; ....
def clean_saihebro(file):
    with open(file, 'r+') as f:
        csv = f.read().replace('  ', '').replace(' ;', ';')
        f.seek(0)
        f.write(csv)
        f.truncate()
#==================================================================================================================
# Function to combine the parameters stationwise
# Always check the column headers, mostlikely the clean function will skip some whitespaces. CTRL + F is your friend.
def combine(array_stations,array_parameter,path,start_date="1998-01-01",end_date="2021-12-31"):
    # Generate timespan as an array
    times = pd.date_range(start=start_date,end=end_date).to_list()
    # Iterate through the text-file stations
    for station in array_stations:
        # Initiate parameter array, all used parameters for the station will be stored here (used in file-naming)
        parameter_array = [] 
        # Initiate Pandas dataframe with timespan as first index column (key for later join operations)
        df_main = pd.DataFrame(data=times,columns=['date'])
        for file in array_parameter:
            if file[1]==station:
                clean_saihebro(file[0]) # Call clean_saihebro function
                df = pd.read_csv(file[0], encoding = 'latin', sep = ';') # Read csv-file as dataframe
                # ------------------------------------------------------------------------------------------------
                # Dataframe cleaning via parameter:
                if file[2]=='TEMPE':
                    df.drop(['Fecha máximo','Fecha mínimo'], axis=1,inplace=True) # Drop redundant columns
                    df.rename(columns={"Máximo": "T_max","Mínimo": "T_min","Media": "T_mean", "fecha ": "date"}, inplace=True)
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    df['T_max'] = df['T_max'].str.replace(',', '.') # Change punctuation to international
                    df['T_min'] = df['T_min'].str.replace(',', '.') # Change punctuation to international
                    df['T_mean'] = df['T_mean'].str.replace(',', '.') # Change punctuation to international
                    df['T_max'] = df['T_max'].astype(float) # Change datatype to float
                    df['T_min'] = df['T_min'].astype(float) # Change datatype to float
                    df['T_mean'] = df['T_mean'].astype(float) # Change datatype to float
                # ------------------------------------------------------------------------------------------------
                if file[2]=='PACUM':
                    df.drop(['Fecha acumulado','Máximo','Fecha máximo'], axis=1,inplace=True) # Drop redundant columns
                    df.rename(columns={"Acumulado": "precipitation", "fecha": "date"}, inplace=True) # Rename columns
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    df['precipitation'] = df['precipitation'].str.replace(',', '.') # Change punctuation to international
                    df['precipitation'] = df['precipitation'].astype(float) # Change datatype to float
                # ------------------------------------------------------------------------------------------------
                if file[2]=='QRIO1' or file[2]=='QRIO2':
                    df.drop(['Fecha máximo','Fecha mínimo'], axis = 1, inplace = True) # Drop redundant columns
                    df.rename(columns={"Máximo": "discharge_max","Mínimo": "discharge_min","Media": "discharge_mean", "fecha ": "date"}, inplace=True) # Rename columns
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    df['discharge_max'] = df['discharge_max'].str.replace(',', '.') # Change punctuation to international
                    df['discharge_min'] = df['discharge_min'].str.replace(',', '.') # Change punctuation to international
                    df['discharge_mean'] = df['discharge_mean'].str.replace(',', '.') # Change punctuation to international
                    df['discharge_max'] = df['discharge_max'].astype(float) # Change datatype to float
                    df['discharge_min'] = df['discharge_min'].astype(float) # Change datatype to float
                    df['discharge_mean'] = df['discharge_mean'].astype(float) # Change datatype to float
                    
                    df.drop(['discharge_max','discharge_min'], axis = 1, inplace = True) # Drop redundant columns
                # ------------------------------------------------------------------------------------------------
                if file[2]=='HUMED':
                    df.drop(['Fecha máximo','Fecha mínimo'], axis=1,inplace=True) # Drop redundant columns
                    df.rename(columns={"Máximo": "rel.humidity_max","Mínimo": "rel.humidity_min","Media": "rel.humidity_mean", "fecha ": "date"}, inplace=True) # Rename columns
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    if df['rel.humidity_min'].dtypes != 'int64':
                        df['rel.humidity_min'] = df['rel.humidity_min'].str.replace(',', '.') # Change punctuation to international
                    df['rel.humidity_max'] = df['rel.humidity_max'].str.replace(',', '.') # Change punctuation to international
                    df['rel.humidity_mean'] = df['rel.humidity_mean'].str.replace(',', '.') # Change punctuation to international
                    df['rel.humidity_max'] = df['rel.humidity_max'].astype(float) # Change datatype to float
                    df['rel.humidity_min'] = df['rel.humidity_min'].astype(float) # Change datatype to float
                    df['rel.humidity_mean'] = df['rel.humidity_mean'].astype(float) # Change datatype to float
                    
                    df.drop(['rel.humidity_max','rel.humidity_min'], axis = 1, inplace = True) # Drop redundant columns
                # ------------------------------------------------------------------------------------------------
                if file[2]=='RADIA':
                    df.drop(['Fecha máximo'], axis=1,inplace=True) # Drop redundant columns
                    df.rename(columns={"Máximo": "radiation_max","Media": "radiation_mean", "fecha": "date"}, inplace=True) # Rename columns
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    df['radiation_max'] = df['radiation_max'].str.replace(',', '.') # Change punctuation to international
                    df['radiation_mean'] = df['radiation_mean'].str.replace(',', '.') # Change punctuation to international
                    df['radiation_max'] = df['radiation_max'].astype(float) # Change datatype to float
                    df['radiation_mean'] = df['radiation_mean'].astype(float) # Change datatype to float
                    
                    df.drop(['radiation_max'], axis = 1, inplace = True) # Drop redundant columns
                # ------------------------------------------------------------------------------------------------
                if file[2]=='VVIEN':
                    # df.rename(columns={"Serie de tiempo": "date","Valor [m/s]": "windspeed"}, inplace=True)
                    df.rename(columns={"FECHA": "date","VALOR": "windspeed"}, inplace=True)
                    df['windspeed'] = df['windspeed'].str.replace(',', '.') # Change punctuation to international
                    df.replace('---','np.nan', inplace=True)
                    df['windspeed'] = df['windspeed'].astype(float) # Change datatype to float
                    # df = df[df['date']>='03/10/2021'] # Filter for dates later then 02.10.2021
                    df['date'] = pd.to_datetime(df['date']) # Change date datatype to datetime
                    df = df.resample('D', on='date').mean() # Daily aggregation 
                    df.replace('np.nan','-9999.0', inplace=True)
                    df.reset_index(inplace=True) # Reset index so the date isn't the index
                # ------------------------------------------------------------------------------------------------
                if file[2]=='INSOL':
                    df.drop('Fecha acumulado', axis = 1, inplace=True)
                    df.rename(columns={"fecha ": "date","Acumulado": "sun_duration"}, inplace=True)
                    # df['sun_duration'] = df['sun_duration'].str.replace(',', '.') NOT NEEDED cause of integer
                    df['date'] = pd.to_datetime(df['date'])
                    df.replace('---','-9999.0', inplace=True)
                    df.replace('np.nan','-9999.0', inplace=True)
                    df['sun_duration'] = df['sun_duration'].astype(float)
                    df.loc[df['sun_duration'] > 0, 'sun_duration'] = df['sun_duration']/3600
                # ------------------------------------------------------------------------------------------------
                parameter_array.append(file[2]) # Append parameter to parameter_array
                df_main = pd.merge(df_main, df, on='date', how='left') # Merge the dataframe to the mainframe
            else: continue # If not equal, ignore the station
        # ------------------------------------------------------------------------------------------------
        df_main.fillna(-9999, inplace=True) # Fill NaN-Values with -9999,0
        # Save the mainframe as csv-file
        df_main.to_csv(r'{}\{}_{}.csv'.format(path,station,'_'.join(parameter_array)), index=False, sep='\t')  
    
    return df_main
#==================================================================================================================
# Function to delete a specific row in multiple csv files. Specify the filepath and the row you want to delete.
def delete_row(path, row_number):
    os.chdir(path) # changing active path
    file_list = glob.glob('*.{}'.format('csv')) # List all csv files in the specified path
    for file in file_list: # Iterate through all the listed files
        df = pd.read_csv(file, encoding = 'latin', sep = '\t') # Read the file as dataframe
        df.drop(row_number, axis = 0, inplace = True) # In case of the April-data this is the 2021-12-31 date row, delete it
        df.to_csv(file, index = False, sep = '\t') # Resave the file
#==================================================================================================================
# Function to create .csv Files for SAIHebro parameterwise
# Necessary for the export to run JAMS
# Also adds the required stationinfo, you must provide them as a txtfile with pathcoord
    # #date A061 A062 ..
    # #date A061 A062 ..
    # elevation 213 213 ..
    # x 100 100 ..
    # y 123 123 ..
# Requirments:
    # Stationswise timeseries data, exported in function combine
    # stationinfo (ID, elevation, x,y (in UTM epsg 32630 for aragon))
def parameter_dat(path,start_date,end_date,pathcoord,outpath):
    os.chdir(path) # changing active path
    file_list = glob.glob('*.{}'.format('csv'))
    times = pd.date_range(start=start_date,end=end_date).to_list()
    # create empty dfs to store all the stations
    df_dis = pd.DataFrame(data=times,columns=['#date'])
    df_precip= pd.DataFrame(data=times,columns=['#date'])
    df_Tmax= pd.DataFrame(data=times,columns=['#date'])
    df_Tmin= pd.DataFrame(data=times,columns=['#date'])
    df_Tmean= pd.DataFrame(data=times,columns=['#date'])
    df_rhum= pd.DataFrame(data=times,columns=['#date'])
    df_wind= pd.DataFrame(data=times,columns=['#date'])
    df_sun= pd.DataFrame(data=times,columns=['#date'])
    # iterate through the stationwise files
    for file in file_list:
        parameters = file.split('_')[1:-2] # extract the parameters messured on this station
        station = file.split('_')[0] # extract the station
        df = pd.read_csv(file, encoding = 'latin', sep = '\t') # open the file
        df['#date'] = pd.to_datetime(df['#date']) # change #date to datetime format
        for parameter in parameters: #iterate through the meassured parameters
            # discharge
            if parameter == 'QRIO1':
                df_x = df[['#date','discharge_mean']].copy() # create new frame with the needed data
                df_x.rename(columns={"discharge_mean": "{}".format(station)}, inplace=True) # rename the parameter to stationID
                df_dis = pd.merge(df_dis, df_x , on='#date', how='left') # store the station in the main parameterframe
            # precipitation
            if parameter == 'PACUM':
                df_x = df[['#date','precipitation']].copy()
                df_x.rename(columns={"precipitation": "{}".format(station)}, inplace=True)
                df_precip = pd.merge(df_precip, df_x , on='#date', how='left')
            # temperature
            if parameter == 'TEMPE':
                # split in min, max, mean
                df_x = df[['#date','T_max']].copy()
                df_x.rename(columns={"T_max": "{}".format(station)}, inplace=True)
                df_Tmax = pd.merge(df_Tmax, df_x , on='#date', how='left')

                df_x = df[['#date','T_min']].copy()
                df_x.rename(columns={"T_min": "{}".format(station)}, inplace=True)
                df_Tmin = pd.merge(df_Tmin, df_x , on='#date', how='left')

                df_x = df[['#date','T_mean']].copy()
                df_x.rename(columns={"T_mean": "{}".format(station)}, inplace=True)
                df_Tmean = pd.merge(df_Tmean, df_x , on='#date', how='left')
            # humidity
            if parameter == 'HUMED':
                df_x = df[['#date','rel.humidity_mean']].copy()
                df_x.rename(columns={"rel.humidity_mean": "{}".format(station)}, inplace=True)
                df_rhum = pd.merge(df_rhum, df_x , on='#date', how='left')
            # windspeed
            if parameter == 'VVIEN':
                df_x = df[['#date','windspeed']].copy()
                df_x.rename(columns={"windspeed": "{}".format(station)}, inplace=True)
                df_wind = pd.merge(df_wind, df_x , on='#date', how='left')
            # insolation
            if parameter == 'INSOL':
                df_x = df[['#date','sun_duration']].copy()
                df_x.rename(columns={"sun_duration": "{}".format(station)}, inplace=True)
                df_sun = pd.merge(df_sun, df_x , on='#date', how='left')
            # skip this extras
            if parameter == 'NEMBA':
                continue
            if parameter == 'VEMBA':
                continue
            if parameter == 'QCAU1':
                continue
    # assign dataframes name (merge resets the name) like the JAMS input .dat files
    df_dis.name='orun'
    df_precip.name='rain'
    df_Tmax.name='tmax'
    df_Tmin.name='tmin'
    df_Tmean.name='tmean'
    df_rhum.name='rhum'
    df_wind.name='wind'
    df_sun.name='sunh'
    # store all frames in a list (easier to handle otherwise subfunctions would be needed)
    frames = [df_dis,df_precip,df_Tmax,df_Tmin,df_Tmean,df_rhum,df_wind,df_sun]

    # iterate through the dataframes
    for frame in frames:
        # read the stationinfos from txt file
        dfcoord = pd.read_csv(pathcoord, encoding = 'latin', sep = '\t')
        # correct the nan values to -9999 for JAMS
        frame.replace(np.nan,-9999.0,inplace=True)
        # dfcoord should contain all the stations in the studyarea BUT not all of them will record all the params, 
        # detect the missing ones and drop them
        not_matching = list(set(dfcoord.columns).difference(frame.columns))
        dfcoord.drop(not_matching , axis = 1 , inplace = True)    
        dfcoord.replace("#date","ID",inplace=True) # rename #date to ID (required by JAMS)
        # assign a new dictionary with 1-2 keys for the dfcoord and the frame
        frames_coord = {"2": frame, "1": dfcoord}
        # merge dfcoord and frame. keys needed so you can adjust the order
        result = pd.concat(frames_coord, keys=["1", "2"])
        # GIMMICK TIME
        # Jams requires a row with: name stat1 stat2 stat3... for each station
        # first fill a list with accordingly n numbers
        number = list(range(0, len(frame.columns)))
        for i in range(len(number)):
            # add the string "stat" infront of the number
            number[i] = 'stat'+ str(number[i])
        # assign the string "name" as the first element in the list
        number[0]='name'
        # rename the column headers by resulting list
        result.set_axis(number, axis=1, inplace=True)
        # Save the csv
        result.to_csv(r'{}\{}.csv'.format(outpath,frame.name), index = False, sep = '\t')
#==================================================================================================================
# Function to rename multiple files. Specific for downloaded SAIHEbro files.
# Example:
# DatosHistoricos_218844_EM25T05VVIEN.csv     to     EM25T05VVIEN.csv
def rename_csv(filelist):
    for file in filelist: # Iterate through the files
        if len(file) == 39: # If the length of the filename is 39:
            new = file.split('_')[-1] # Split the filenames by '_' and keep the last part of it
            os.rename(file, new) # Rename the file rename(old name, new name)
    print('Files renamed...')
#==================================================================================================================
# Function to combine 2 .csv Files into 1 large .csv
# Requirements:
    # Identical filenames with different endings in one folder. EXAMPLE:
        # ..\Year_Combi\0_raw\A061_QRIO1_PACUM_tab.csv
        # ..\Year_Combi\0_raw\A061_QRIO1_PACUM_2022.csv
    # Date column should be named #date (like RBIS is accepting the first row in the file) otherwise change the code or
        # you can use function add_firstsign as a prep approach
def year_combi(path):
    os.chdir(path) # changing active path
    file_list = glob.glob('*.{}'.format('csv')) # List all csv files in the specified path
    for i in range(0,len(file_list)-1,2): # Iterate through each 2n element in range of the length of  filelist -1 
        df_old = pd.read_csv(file_list[i+1], encoding = 'latin', sep = '\t') # open the i+1 element (2/2 Element of the pair)
        df_old.rename(columns={"# date": "#date"}, inplace=True) # Rename 
        df_old['#date'] = pd.to_datetime(df_old['#date'])

        df_new = pd.read_csv(file_list[i], encoding = 'latin', sep = '\t') # open the i element (1/2 Element of the pair)
        df_new.rename(columns={"# date": "#date"}, inplace=True)
        df_new['#date'] = pd.to_datetime(df_new['#date'])

        frames = [df_old,df_new]
        result = pd.concat(frames)
        result.to_csv(r'{}\done\{}_combi.csv'.format(path,file_list[i+1].split('.')[0]), index=False, sep='\t')
#==================================================================================================================

In [54]:
# Main
#==================================================================================================================
# Insert your path to your text- and .csv files
path  = r'E:\1_Data_process\4_timeseries\SAIHebro\april_data\0_basedata'
#start_date = '2021-12-31'
#end_date = '2022-04-30'
start_date = '1998-01-01'
end_date = '2021-12-31'
os.chdir(path) # Change directory 
txt_lists = glob.glob('*.{}'.format('txt')) # List all subfiles in the path
for i in range(len(txt_lists)):
    txt_lists[i] = path+'\\'+txt_lists[i] # Append the fullpath to txt_lists array
# Call the array_stations function to extract all station numbers from one text-file and list all station numbers
all_stations = array_stations(txt_lists[0])
#==================================================================================================================
time_series_list = glob.glob('*.{}'.format('csv'))
rename_csv(time_series_list)
# Append the fullpath to time_s]eries_list array, add the station number 
for i in range(len(time_series_list)):
    time_series_list[i] = [path+'\\'+time_series_list[i],time_series_list[i][:4],time_series_list[i][7:-4]]
#==================================================================================================================
# Check if the stations of the listed csv-files are equal to the stations listed in the text-files
all_parameter = [] # Array to append the matching stations
#==================================================================================================================
for i in time_series_list: # Iterate through the listed csv-files
    if i[1] in all_stations: # Equal with text-file ?
        all_parameter.append(i) # If equal append them to all_paramter
#==================================================================================================================
output_path = r'E:\1_Data_process\4_timeseries\SAIHebro\april_data\1_done'
df = combine(all_stations,all_parameter,output_path,start_date,end_date) # Call the combine function
#==================================================================================================================
add_firstsign(output_path)

Files renamed...


In [5]:
output_path = r'E:\1_Data_process\4_timeseries\SAIHebro\EM_remake\1998'
add_firstsign(output_path)

Added # in first row...
